# Use case introduction 📰

In this notebook, we'll explore the use case of extracting structured information from news articles.

In short, given a news article, the goal is to extract the following information:
- The title of the article
- A summary of the article
- Whether the article is about business
- For each mentioned business
  - The name of the business
  - The likely stock price change (increase, decrease, none)
  - Reason for the stock price change
  - Relevant substring supporting the reason


## Exploring the data

We are using a dataset of BBC news articles for a specific month (see `get_bbc_news_sample` function below). The dataset contains the following columns: 
- `article`: The article as we would see it on the website.
- `title`: The title of the article.
- `is_business`: Whether the article is about business.
- `description`: A short summary of the article.

As you see, we start with a *labeled* data set, as is important for many use cases. If this is not available in your case, it may be worth to label a few samples to get started.

In [13]:
from llmops_training.news_reader.data import get_bbc_news_sample
from rich import print

In [2]:
data = get_bbc_news_sample()
data

,article,is_business,description,title
0,Unions pressure ministers to increase workers'...,False,A number of union leaders have written to the ...,Unions pressure ministers to increase workers'...
1,Syria in maps: Who controls the country now As...,False,A visual guide to the latest developments afte...,Syria in maps: Who controls the country now As...
2,Turkey's 3m Syrian refugees face big decision ...,False,Thousands of Syrians have flocked to Turkey's ...,Turkey's 3m Syrian refugees face big decision ...
3,"Housing plans not war on rural England, says m...",False,Matthew Pennycook says development will be foc...,"Housing plans not war on rural England, says m..."
4,BBC 100 Women: Sharon Stone’s emotional messag...,False,The film star opens up to the BBC about nearly...,BBC 100 Women: Sharon Stone’s emotional messag...
...,...,...,...,...
137,Thames Water boss defends bonuses as sewage sp...,True,The company saw a 40% increase in pollution in...,Thames Water boss defends bonuses as sewage sp...
138,Guinness supplies limited ahead of Christmas a...,True,Guinness-maker Diageo has placed limits on who...,Guinness supplies limited ahead of Christmas a...
139,Stellantis boss Carlos Tavares abruptly quits ...,True,"Carlos Tavares will leave immediately, days af...",Stellantis boss Carlos Tavares abruptly quits ...
140,Avanti West Coast strike at Christmas called o...,True,Train managers will vote on a revised offer fr...,Avanti West Coast strike at Christmas called o...


Let's print the first article which is about business:

In [3]:
article = data[data["is_business"]].iloc[0].article
print(article)

Stocks slide as US central bank signals slower pace of rate cuts - BBC News

Stocks slide as US central bank signals slower pace of rate cuts
US share prices slumped after the central bank cut interest rates for the third time in a row but its economic projections signalled a slower pace of cuts next year. In a widely expected move, the Federal Reserve set its key lending rate in a target range of 4.25% to 4.5%. That is down a full percentage point since September, when the bank started lowering borrowing costs, citing progress stabilising prices and a desire to head off economic weakening. Reports since then indicate that the number of jobs being created has been more resilient than expected, while price rises have continued to bubble.
Stocks in the US fell sharply as Federal Reserve chairman Jerome Powell warned the situation would likely result in fewer rate cuts than expected next year. "We are in a new phase of the process," he said at a press conference. "From this point forward,

## Defining the LLM output

Now, let's define the Pydantic schema for the LLM to extract from each article.

> **Exercise** 📝
>
> - Fill in the TODO's below in the `ArticleInfo` Pydantic model.
> - Subsequently, fill in the prompt in the next section to extract the information from an article.
>

In [4]:
from typing import List, Literal
from pydantic import BaseModel, Field

In [5]:
class BusinessSpecificInfo(BaseModel):
    business: str = Field(..., description="The business or company involved")
    stock_price_change: Literal["increase", "decrease", "none"] = Field(
        ..., description="""
        Possible stock price change as result of the article. 
        - "increase" if article speaks positively about the business
        - "decrease" if article speaks negatively about the business
        - "none" if article speaks neutrally about the business
        """
    )
    reason: str = Field(..., description="A single sentence reason for the possible stock price change")
    relevant_substring: str = Field(
        ..., description="A relevant substring from the article supporting the reason (10-20 words)"
    )

class ArticleInfo(BaseModel):
    title: str = Field(..., description="The title of the article")
    summary: str = Field(..., description="A single sentence summary of the article")
    is_about_business: bool = Field(..., description="Whether the article is about business")
    business_info: List[BusinessSpecificInfo] = Field(..., description="For each business mentioned in the article, the relevant information")
    #is_about_business: ...  # TODO: Fill me in! This should be a boolean (use Field here)
    #business_info: ...  # TODO: Fill me in! This should be a List of BusinessSpecificInfo objects (no need to use Field here)

In [8]:
# %load ../solutions/use-case-introduction/pydantic-model.py


## Extracting the information

In [7]:
from llmops_training.news_reader.generation import generate_object

In [9]:
prompt_template = "Extract structured information from the following article:\n\n{article}"

prompt = prompt_template.format(article=article)  # TODO: Fill me in! Use the .format function on prompt_template with an article of your choice


In [ ]:
# %load ../solutions/use-case-introduction/make-prompt.py

Let's see if that worked!

In [15]:
output = generate_object(prompt, ArticleInfo)

In [16]:
print(output)

ArticleInfo(
    title='Stocks slide as US central bank signals slower pace of rate cuts - BBC News',
    summary="US share prices fell sharply following the Fed's decision to cut rates, as signals of fewer future 
cuts raised concerns over inflation and economic outlook.",
    is_about_business=True,
    business_info=[
        BusinessSpecificInfo(
            business='Federal Reserve',
            stock_price_change='decrease',
            reason='Its announcement signaled a slower pace of rate cuts, triggering a sharp decline in US stock 
markets.',
            relevant_substring='US stocks fell sharply as Jerome Powell warned fewer rate cuts.'
        ),
        BusinessSpecificInfo(
            business='Bank of England',
            stock_price_change='none',
            reason='The Bank of England is expected to hold its benchmark rate steady despite inflation 
pressures.',
            relevant_substring='the Bank of England is due to make its latest interest rates decision in the UK'
        )
    ]
)

In [14]:
output.model_dump()

{'title': 'Stocks slide as US central bank signals slower pace of rate cuts - BBC News',
 'summary': "US stocks fell sharply after the Federal Reserve's rate cut and signals of fewer future cuts, causing major indices to drop both in the US and Asia.",
 'is_about_business': True,
 'business_info': [{'business': 'Dow Jones Industrial Average',
   'stock_price_change': 'decrease',
   'reason': 'Investors panicked after Fed signaled fewer future rate cuts, marking a prolonged decline.',
   'relevant_substring': 'The Dow Jones Industrial Average closed 2.58% lower, suffering its 10th session of declines in a row'},
  {'business': 'S&P 500',
   'stock_price_change': 'decrease',
   'reason': "The Fed's cautious signal on future rate cuts spooked investors, reflecting in a roughly 3% fall.",
   'relevant_substring': 'The S&P 500 lost almost 3% and the Nasdaq Composite fell 3.6%'},
  {'business': 'Nasdaq Composite',
   'stock_price_change': 'decrease',
   'reason': 'Concerns over a slower pace

Extra: check `generate_object` and figure out how to try different models! Try some different ones and see which one works well for this task.

> Tip: Use `model_name` and try for example `gpt-5`.

## (Bonus) Deep dive Instructor

> **Exercise** 📝
>
> - Step into the underlying code from `generate_object` and try to understand how Instructor extracts structured information from the text given a Pydantic base model.
> - Does it use [Function Calling](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling)? Does it do retry iterations? Could we implement this ourselves? Discuss with your peers!

---